In [2]:
from rdkit import Chem
import pandas as pd

C:\Users\Chaewon\AppData\Local\Temp\ipykernel_8928\853024002.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
date = '0216'

In [4]:

df = pd.read_csv(f'GPT_response_final_result_{date}.csv')
Lowe_smiles = df['Lowe_smiles'].tolist()
GPT_smiles = df['GPT_smiles'].tolist()
titles = df['nameResolved'].tolist()
paragraphs = df['paragraph'].tolist()
responses = df['json_responses'].tolist()
df.head(3)

,Unnamed: 0.1,Unnamed: 0,processed_random_samples_Index,temp_compare_Index,documentId,paragraphNum,file_path,headingText,nameResolved,template,productSmiles,paragraph,Lowe_smiles,GPT response,json_responses,smiles_dict,smiles_dict_fixed,GPT_smiles
0,0,0,15,1835,US06093732,NaN,C:/1976_Sep2016_USPTOgrants_cml/grants/2000/pf...,N-(4-Chlorobenzyl)-4-hydroxy-6-(3-hydroxy-1-pr...,N-(4-Chlorobenzyl)-4-hydroxy-6-(3-hydroxy-1-pr...,[C:1]#[C:2].[X:6]-[c:3](:[a:4]):[a:5]>>[C:1]#[...,COc1cc(C#CCO)cc2c(O)c(C(=O)NCc3ccc(Cl)cc3)cnc12,N-(4-Chlorobenzyl)-4-hydroxy-6-iodo-8-methoxy-...,[Cl:1][C:2]1[CH:25]=[CH:24][C:5]([CH2:6][NH:7]...,"\n {\n ""Detailed Reaction Description"": ...","{'Detailed Reaction Description': {'Reactants,...",{'A': 'COc1cc(I)cc2c(O)c(cnc12)C(=O)NCc3ccc(Cl...,{'A': 'COc1cc(I)cc2c(O)c(cnc12)C(=O)NCc3ccc(Cl...,COc1cc(I)cc2c(O)c(cnc12)C(=O)NCc3ccc(Cl)cc3.OC...
1,1,1,188,30257,US20140315881A1,927.0,C:/2001_Sep2016_USPTOapplications_cml/applicat...,a) methyl 2-bromo-2-(2-bromobenzofuran-5-yl)ac...,methyl 2-bromo-2-(2-bromobenzofuran-5-yl)acetate,[C:1].O=C1-C-C-C(=O)-[N:3]-1-[Br:2]>>[C:1]-[Br:2],COC(=O)C(Br)c1ccc2oc(Br)cc2c1,To a solution of methyl 2-(2-bromobenzofuran-5...,[Br:1][C:2]1[O:3][C:4]2[CH:10]=[CH:9][C:8]([CH...,"{\n ""Detailed Reaction Description"": {\n ""...","{'Detailed Reaction Description': {'Reactants,...","{'A': 'COC(=O)Cc1ccc2oc(Br)cc2c1', 'B': 'ClC(C...","{'A': 'COC(=O)Cc1ccc2oc(Br)cc2c1', 'B': 'ClC(C...",COC(=O)Cc1ccc2oc(Br)cc2c1.BrN1C(=O)CCC1=O.CC(C...
2,2,2,167,891,US05354749,NaN,C:/1976_Sep2016_USPTOgrants_cml/grants/1994/pf...,4-(Bromomethyl)benzene-sulphonyl chloride ##ST...,4-(Bromomethyl)benzene-sulphonyl chloride,[C:1].O=C1-C-C-C(=O)-[N:3]-1-[Br:2]>>[C:1]-[Br:2],O=S(=O)(Cl)c1ccc(CBr)cc1,38.1 g (0.2 mol) of 4-methylbenzenesulphonyl c...,[CH3:1][C:2]1[CH:7]=[CH:6][C:5]([S:8]([Cl:11])...,"{\n ""Detailed Reaction Description"": {\n ""...","{'Detailed Reaction Description': {'Reactants,...","{'A': 'Cc1ccc(cc1)[S](Cl)(=O)=O', 'B': 'ClC(Cl...","{'A': 'Cc1ccc(cc1)[S](Cl)(=O)=O', 'B': 'ClC(Cl...",Cc1ccc(cc1)[S](Cl)(=O)=O.BrN1C(=O)CCC1=O.CC(C)...


In [5]:
def demap(smiles, return_role=False):
    mol = Chem.MolFromSmiles(smiles)
    is_reactant = False
    if mol is None:  # Check if the molecule is valid
        return (smiles, False) if return_role else smiles  # Return the original SMILES if invalid

    for atom in mol.GetAtoms():
        if atom.GetAtomMapNum() != 0:
            is_reactant = True #reactant이다
            atom.SetAtomMapNum(0)
    if return_role: #return_role = True인 경우
        # print('is_reactant', is_reactant)
        return Chem.MolToSmiles(mol), is_reactant
    else: # return_role = False인 경우
        return Chem.MolToSmiles(mol)
        
def check_NoSmi(string1, string2, string3):
    check_NoSmi = False
    all_string = string1+string2+string3
    if 'NoSmi' in all_string:
        check_NoSmi = True
    return check_NoSmi

def split_role(rxn):
    reactants, reagents, product = rxn.split('>')
    if not check_NoSmi(reactants,reagents,product):
        if not reagents:
            real_reactants, real_reagents = [], []
        else:
            real_reactants, real_reagents = [], [demap(reagents)]
        for reactant in reactants.split('.'):
            smiles, is_reactant = demap(reactant, True)
            if is_reactant:
                real_reactants.append(smiles)
            else:
                real_reagents.append(smiles)
        return real_reactants, real_reagents, demap(product)
    else:
        print('NoSmi exists in rxn')
        return None
    


In [13]:
global Error_cnt
Error_cnt = 0

def compare_results(i):
    global Error_cnt
    try:
        title = titles[i]
        paragraph = paragraphs[i]
        response = responses[i]
        rxn1 = Lowe_smiles[i]
        rxn2 = GPT_smiles[i]

        reactants1, reagents1, products1 = split_role(rxn1)
        reactants2, reagents2, products2 = split_role(rxn2)

        rxn1_formatted = '.'.join(sorted(reactants1 + reagents1)) + '>>' + '.'.join(sorted(products1))
        rxn2_formatted = '.'.join(sorted(reactants2 + reagents2)) + '>>' + '.'.join(sorted(products2))

        result = {
            'title': title,
            'paragraph': paragraph,
            'response': response,
            'Lowe_rxn': rxn1_formatted,
            'GPT_rxn': rxn2_formatted,
            'reactant_match': set(reactants1) == set(reactants2),
            'reagent_match': set(reagents1) == set(reagents2),
            'product_match': set(products1) == set(products2),
            'rxn_match': rxn1_formatted == rxn2_formatted
        }
        return result
    except Exception as e:
        Error_cnt += 1
        # result = {'Error': str(e)}
        print(f'Error at index {i}: {e}')
        return None

    

# Assuming titles, paragraphs, responses, Lowe_smiles, GPT_smiles are defined elsewhere
i = 2
results = compare_results(i)
results
# print(Error_cnt)


{'title': '4-(Bromomethyl)benzene-sulphonyl chloride',
 'paragraph': '38.1 g (0.2 mol) of 4-methylbenzenesulphonyl chloride are dissolved in 300 ml of carbon tetrachloride, 35.6 g (0.2 mol) of N-bromosuccinimide are added and, after addition of 0.2 g (1.2 mmol) of azobisisobutyronitrile (AIBN),the mixture is heated under reflux for 4 hours. After cooling, the solids are filtered off and the filtrate is freed from the solvent. Flash chromatography (petroleum ether/toluene 4:1, 50 μm particle size) and subsequent recrystallisation from 100 ml of cyclohexane give 24.0 g (45% of theory) of the title compound. Rf =0.75 (toluene).',
 'response': "{'Detailed Reaction Description': {'Reactants, Solvents, and Catalysts': {'A': '4-Methylbenzenesulphonyl chloride', 'B': 'Carbon tetrachloride', 'C': 'N-Bromosuccinimide', 'D': 'Azobisisobutyronitrile'}, 'Products': {'E': '4-(Bromomethyl)benzene-sulphonyl chloride'}, 'Reaction Steps': {'1': '(Dissolve, Room Temperature, None, None): A + B -> Solutio

In [18]:
from tqdm import tqdm

match_cnt = {'product':[], 'reactant':[], 'reagent':[], 'rxn':[]}
no_rxns = []
for i in tqdm(range(len(df)), total=len(df)):
    result = compare_results(i)
    # if 'NoSmi' in result['GPT_rxn']:
    #     no_rxns.append(i)
        # continue
    if result == None:
        no_rxns.append(i)
        continue

    for k in match_cnt.keys():
        if not result['%s_match' % k]: #False인 경우, GPT랑 Lowe랑 결과가 다른 경우에 match_cnt에 append, 다른 경우가 많아야 좋음, 즉 숫자 높을수록 좋음
            # print(f'{i} result')
            # print(f'{k}_match',result['%s_match' % k])
            match_cnt[k].append(i)
        
#     if not result['GPT_rxn']:
#         match_cnt['no_rxn'].append(i)
        


  0%|          | 0/54 [00:00<?, ?it/s]

[15:48:15] WARNING: not removing hydrogen atom without neighbors
[15:48:15] WARNING: not removing hydrogen atom without neighbors
[15:48:15] WARNING: not removing hydrogen atom without neighbors
[15:48:15] WARNING: not removing hydrogen atom without neighbors
[15:48:15] WARNING: not removing hydrogen atom without neighbors
[15:48:15] WARNING: not removing hydrogen atom without neighbors
[15:48:15] WARNING: not removing hydrogen atom without neighbors
[15:48:15] SMILES Parse Error: syntax error while parsing: [Pd](|OC(C)=O)|OC(C)=O
[15:48:15] SMILES Parse Error: Failed parsing SMILES '[Pd](|OC(C)=O)|OC(C)=O' for input: '[Pd](|OC(C)=O)|OC(C)=O'
[15:48:15] SMILES Parse Error: syntax error while parsing: [Pd](|OC(C)=O)|OC(C)=O
[15:48:15] SMILES Parse Error: Failed parsing SMILES '[Pd](|OC(C)=O)|OC(C)=O' for input: '[Pd](|OC(C)=O)|OC(C)=O'
[15:48:15] WARNING: not removing hydrogen atom without neighbors
[15:48:15] WARNING: not removing hydrogen atom without neighbors
100%|██████████| 54/54 

NoSmi exists in rxn
Error at index 15: cannot unpack non-iterable NoneType object
NoSmi exists in rxn
Error at index 24: cannot unpack non-iterable NoneType object
NoSmi exists in rxn
Error at index 28: cannot unpack non-iterable NoneType object
NoSmi exists in rxn
Error at index 45: cannot unpack non-iterable NoneType object
NoSmi exists in rxn
Error at index 49: cannot unpack non-iterable NoneType object


In [22]:
# print(match_cnt)
# print(Error_cnt)
print('no_rxns:', len(no_rxns))
print('product:', match_cnt['product'])
print('total data number', len(df))
print('NoSmi:', len(no_rxns)/len(df)*100, '%')

no_rxns: 5
product: [39]
total data number 54
NoSmi: 9.25925925925926 %


In [23]:
print('No smi:', '(%.2f)' % (len(no_rxns)/len(df)))
print ()
for k, v in match_cnt.items():
    vl = len(v)
    print (k, vl, '(%.2f)' % (vl/len(df)), v[:20])

No smi: (0.09)

product 1 (0.02) [39]
reactant 49 (0.91) [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20]
reagent 49 (0.91) [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20]
rxn 32 (0.59) [0, 1, 2, 3, 6, 12, 13, 14, 17, 18, 19, 20, 22, 23, 25, 27, 29, 30, 31, 32]
